# EDA Univariado y Bivariado – HEXAD Study 2

_Objetivo:_ Explorar distribuciones (edad, género, países), porcentajes de motivaciones Hexad y relaciones básicas (edad/género vs rasgos). Incluir fiabilidad (alfa de Cronbach) para sustentar puntajes compuestos.

## Preguntas guía
1. ¿La **edad** se relaciona con alguna **motivación Hexad**?
2. ¿Existen **diferencias por género** en los porcentajes de motivación?
3. ¿Qué **rasgos co-varían** entre sí?
4. ¿La **fiabilidad** de los ítems por rasgo es suficiente?


## 0) Configuración
- Ejecuta las celdas en orden.
- Coloca tu CSV en `data/dataset-hexad-12-study-2.csv` (no lo subas si es privado).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

PLOTS_DIR = Path('eda_plots'); PLOTS_DIR.mkdir(parents=True, exist_ok=True)
DATA_PATH = Path('data') / 'dataset-hexad-12-study-2.csv'
print('Carpetas listas. DATA_PATH =', DATA_PATH)

## 1) Carga y vista general

In [ ]:
try:
    df = pd.read_csv(DATA_PATH)
    print('✔ Dataset cargado:', DATA_PATH)
except Exception as e:
    raise SystemExit('⚠ No pude cargar el CSV. Asegúrate de que exista en data/.\nError: ' + str(e))

display(df.head())
print('Dimensiones (filas, columnas):', df.shape)
display(df.dtypes)

## 2) Calidad de datos y saneamiento

In [ ]:
na = df.isna().sum().sort_values(ascending=False)
na_pct = (na/len(df)*100).round(2)
display(pd.DataFrame({'faltantes': na, '%': na_pct}).head(15))

df['age_num'] = pd.to_numeric(df.get('age'), errors='coerce')
print('age_num válidos:', df['age_num'].notna().sum())

## 3) Univariado: demografía y rasgos

In [ ]:
display(df['age_num'].describe())
display(df['gender'].value_counts(dropna=False))
display(df['countryOfOrigin_label'].value_counts().head(10))

plt.figure(); df['age_num'].dropna().plot(kind='hist', bins=30)
plt.title('Distribución de edad'); plt.xlabel('Edad'); plt.ylabel('Frecuencia')
plt.savefig(PLOTS_DIR / 'hist_edad.png', bbox_inches='tight'); plt.show()

plt.figure(); df['gender'].value_counts(dropna=False).plot(kind='bar')
plt.title('Frecuencia por género'); plt.xlabel('Género'); plt.ylabel('Conteo')
plt.savefig(PLOTS_DIR / 'bar_genero.png', bbox_inches='tight'); plt.show()

perc_cols = [c for c in df.columns if c.startswith('hexadType_percentage_')]
display(df[perc_cols].describe().T)
for c in perc_cols:
    plt.figure(); df[c].dropna().plot(kind='hist', bins=20)
    plt.title(f'Distribución: {c}'); plt.xlabel('Porcentaje (0-100)'); plt.ylabel('Frecuencia')
    plt.savefig(PLOTS_DIR / f'hist_{c}.png', bbox_inches='tight'); plt.show()

## 4) Puntajes por rasgo y fiabilidad (alfa de Cronbach)

In [ ]:
def cronbach_alpha(df_items):
    df_items = df_items.dropna()
    k = df_items.shape[1]
    if k < 2:
        return np.nan
    variances = df_items.var(axis=0, ddof=1)
    total_var = df_items.sum(axis=1).var(ddof=1)
    if total_var == 0:
        return np.nan
    return (k/(k-1))*(1 - variances.sum()/total_var)

traits = {'A': 'achiever', 'D': 'disruptor', 'F': 'freeSpirit', 'P': 'philanthropist', 'R': 'player', 'S': 'socializer'}
items_by_trait = {t: [f"{t}{i}" for i in range(1,5)] for t in traits.keys()}

alphas = {}
for prefix, cols in items_by_trait.items():
    trait = traits[prefix]
    sum_col = f'{trait}_sum4'
    df[sum_col] = df[cols].sum(axis=1, min_count=1)
    alphas[trait] = cronbach_alpha(df[cols])

alphas

## 5) Bivariado: correlaciones y comparaciones por grupos

In [ ]:
targets = perc_cols + [c for c in df.columns if c.endswith('_sum4')]
corr = df[targets].corr(method='pearson')
display(corr.round(2))

plt.figure(figsize=(8,6))
plt.imshow(corr, aspect='auto', interpolation='nearest')
plt.colorbar(); plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.index)), corr.index); plt.title('Matriz de correlaciones (Pearson)')
plt.tight_layout(); plt.savefig(PLOTS_DIR / 'correlaciones_heatmap.png', bbox_inches='tight'); plt.show()

age_corr = df[perc_cols + ['age_num']].corr().loc('age_num') if 'age_num' in df.columns else None
age_corr

## 6) Interpretación (escribe aquí tus conclusiones)
- **Edad vs rasgos**: ¿correlaciones pequeñas/medias? ¿qué ves en los histogramas/heatmap?
- **Género vs rasgos**: ¿diferencias visibles? (agrega boxplots si lo necesitas)
- **Co-variación**: ¿pares de rasgos con correlación más alta?
- **Fiabilidad**: ¿α ≥ 0.70 en la mayoría? ¿advertencias?
